In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import polars as pl # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd #backup

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ing-hubs-turkiye-datathon/sample_submission.csv
/kaggle/input/ing-hubs-turkiye-datathon/referance_data.csv
/kaggle/input/ing-hubs-turkiye-datathon/referance_data_test.csv
/kaggle/input/ing-hubs-turkiye-datathon/customers.csv
/kaggle/input/ing-hubs-turkiye-datathon/customer_history.csv


In [2]:
# Eğer yüklü değilse Polars'ı yükle (bu hücreyi sadece bir kere çalıştır)
!pip install polars --quiet

# Kütüphaneleri içe aktar
import polars as pl
import pandas as pd # Makine öğrenimi modelleri (sklearn, xgboost) genellikle Pandas DataFrame bekler.
import numpy as np # Polars'ta doğrudan kullanılmasa da genel sayısal işlemler için

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
import xgboost as xgb # Makine öğrenimi modelimiz

# Pandas'ın tüm sütunları gösterme ayarları (Polars için geçerli değil ama Pandas'a döndüğümüzde lazım olabilir)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Gerekli kütüphaneler yüklendi ve içe aktarıldı.")

Gerekli kütüphaneler yüklendi ve içe aktarıldı.


# Adım 1: Veri Yükleme ve Birleştirme (Polars)

In [3]:

data_path = '/kaggle/input/ing-hubs-turkiye-datathon/'

# Polars kullanarak veri yükleme ve tarih sütunlarını dönüştürme
customer_history_pl = pl.read_csv(data_path + 'customer_history.csv').with_columns(
    pl.col("date").str.to_date("%Y-%m-%d") # YYYY-MM-DD formatındaki metni Date tipine çevir
)
customers_pl = pl.read_csv(data_path + 'customers.csv')
reference_data_pl = pl.read_csv(data_path + 'referance_data.csv').with_columns(
    pl.col("ref_date").str.to_date("%Y-%m-%d")
)
reference_data_test_pl = pl.read_csv(data_path + 'referance_data_test.csv').with_columns(
    pl.col("ref_date").str.to_date("%Y-%m-%d")
)
sample_submission_pl = pl.read_csv(data_path + 'sample_submission.csv')


print("Veriler Polars DataFrame olarak başarıyla yüklendi.")

# Ana eğitim ve test veri çerçevelerini oluşturalım
# Müşteri demografik bilgilerini birleştirelim (Polars'ta join)
train_df_pl = reference_data_pl.join(customers_pl, on='cust_id', how='left')
test_df_pl = reference_data_test_pl.join(customers_pl, on='cust_id', how='left')

print("customers ve reference_data birleştirildi (Polars).")
print("Train DF (Polars) İlk 5 Satır:")
print(train_df_pl.head())
print("\nTest DF (Polars) İlk 5 Satır:")
print(test_df_pl.head())

Veriler Polars DataFrame olarak başarıyla yüklendi.
customers ve reference_data birleştirildi (Polars).
Train DF (Polars) İlk 5 Satır:
shape: (5, 10)
┌─────────┬────────────┬───────┬────────┬───┬──────────┬────────────┬───────────────┬────────┐
│ cust_id ┆ ref_date   ┆ churn ┆ gender ┆ … ┆ religion ┆ work_type  ┆ work_sector   ┆ tenure │
│ ---     ┆ ---        ┆ ---   ┆ ---    ┆   ┆ ---      ┆ ---        ┆ ---           ┆ ---    │
│ i64     ┆ date       ┆ i64   ┆ str    ┆   ┆ str      ┆ str        ┆ str           ┆ i64    │
╞═════════╪════════════╪═══════╪════════╪═══╪══════════╪════════════╪═══════════════╪════════╡
│ 0       ┆ 2017-09-01 ┆ 0     ┆ F      ┆ … ┆ U        ┆ Part-time  ┆ Technology    ┆ 135    │
│ 3       ┆ 2018-10-01 ┆ 0     ┆ F      ┆ … ┆ C        ┆ Student    ┆ null          ┆ 47     │
│ 5       ┆ 2018-03-01 ┆ 1     ┆ M      ┆ … ┆ U        ┆ Full-time  ┆ Finance       ┆ 108    │
│ 6       ┆ 2018-04-01 ┆ 1     ┆ F      ┆ … ┆ U        ┆ Unemployed ┆ null          ┆ 187 

# Adım 2: Tarih İşlemleri ve Özellik Mühendisliği (Polars - DÜZELTİLMİŞ)


In [4]:

print("\nPolars ile müşteri geçmişi özellik mühendisliği başlatılıyor...")

# Özelliklerini çıkaracağımız sütunlar
hist_cols_to_agg = [
    'mobile_eft_all_cnt', 'mobile_eft_all_amt',
    'cc_transaction_all_amt', 'cc_transaction_all_cnt',
    'active_product_category_nbr'
]

# Lazy çerçeveleri oluşturalım (bellek ve performans optimizasyonu için)
lazy_train_df = train_df_pl.lazy()
lazy_test_df = test_df_pl.lazy()
lazy_customer_history = customer_history_pl.lazy()

def extract_history_features(main_lazy_df, history_lazy_df):
    
    joined_df_lazy = main_lazy_df.join(history_lazy_df, on="cust_id", how="left")\
                                 .filter(pl.col("date") < pl.col("ref_date"))

    # Standart agregasyonlar (mean, sum, max, count)
    agg_expressions = [
        pl.col(col).mean().alias(f"{col}_mean_hist") for col in hist_cols_to_agg
    ] + [
        pl.col(col).sum().alias(f"{col}_sum_hist") for col in hist_cols_to_agg
    ] + [
        pl.col(col).max().alias(f"{col}_max_hist") for col in hist_cols_to_agg
    ] + [
        pl.col("date").count().alias("history_record_count")
    ]
    
    # DÜZELTME BURADA: En son ayın değeri için .over() yerine .sort_by().first() kullanıyoruz
    last_month_features_expr = []
    for col in hist_cols_to_agg:
        last_month_features_expr.append(
            # Grubu tarihe göre tersten sırala ve ilk değeri al (bu en son aydır)
            pl.col(col).sort_by(pl.col("date"), descending=True).first().alias(f"{col}_last_month")
        )

    # Grup bazında aggregasyonları yap ve collect ile Polars DataFrame'e çevir
    features_from_history = joined_df_lazy.group_by("cust_id", "ref_date").agg(
        *agg_expressions,
        *last_month_features_expr  # Düzeltilmiş ifadeyi ekle
    ).sort("cust_id", "ref_date").collect()

    return features_from_history

# Eğitim ve test setleri için özellikleri oluştur
train_hist_features_pl = extract_history_features(lazy_train_df, lazy_customer_history)
test_hist_features_pl = extract_history_features(lazy_test_df, lazy_customer_history)

# Ana DF'lere yeni özellikleri birleştir (Polars)
train_df_pl = train_df_pl.join(train_hist_features_pl, on=['cust_id', 'ref_date'], how='left')
test_df_pl = test_df_pl.join(test_hist_features_pl, on=['cust_id', 'ref_date'], how='left')

print("\nPolars ile müşteri geçmişi özellik mühendisliği tamamlandı.")
print("Train DF (Polars, özelliklerle) İlk 5 Satır:")
print(train_df_pl.head())

# Eksik Değerleri Doldurma (Polars)
print("\nPolars'ta null (NaN) değerler dolduruluyor...")
for col in train_df_pl.columns:
    if train_df_pl[col].dtype.is_numeric():
        if train_df_pl[col].is_null().any():
            mean_val = train_df_pl[col].mean()
            fill_value = mean_val if mean_val is not None and not np.isnan(mean_val) else 0

            train_df_pl = train_df_pl.with_columns(
                pl.col(col).fill_null(fill_value)
            )
            test_df_pl = test_df_pl.with_columns(
                pl.col(col).fill_null(fill_value)
            )

print("\nPolars'ta null (NaN) değerler dolduruldu.")
print("Train DF (Polars) Null kontrolü (ilk 50 sütun):")
print(train_df_pl.null_count().head(50))
print("\nTest DF (Polars) Null kontrolü (ilk 50 sütun):")
print(test_df_pl.null_count().head(50))


Polars ile müşteri geçmişi özellik mühendisliği başlatılıyor...

Polars ile müşteri geçmişi özellik mühendisliği tamamlandı.
Train DF (Polars, özelliklerle) İlk 5 Satır:
shape: (5, 31)
┌─────────┬────────────┬───────┬────────┬───┬─────────────┬─────────────┬─────────────┬────────────┐
│ cust_id ┆ ref_date   ┆ churn ┆ gender ┆ … ┆ mobile_eft_ ┆ cc_transact ┆ cc_transact ┆ active_pro │
│ ---     ┆ ---        ┆ ---   ┆ ---    ┆   ┆ all_amt_las ┆ ion_all_amt ┆ ion_all_cnt ┆ duct_categ │
│ i64     ┆ date       ┆ i64   ┆ str    ┆   ┆ t_month     ┆ _last_mo…   ┆ _last_mo…   ┆ ory_nbr_la │
│         ┆            ┆       ┆        ┆   ┆ ---         ┆ ---         ┆ ---         ┆ …          │
│         ┆            ┆       ┆        ┆   ┆ f64         ┆ f64         ┆ f64         ┆ ---        │
│         ┆            ┆       ┆        ┆   ┆             ┆             ┆             ┆ i64        │
╞═════════╪════════════╪═══════╪════════╪═══╪═════════════╪═════════════╪═════════════╪════════════╡
│ 0   

# Adım 3: Kategorik Değişken İşleme (Polars'tan Pandas'a dönüştürme ve One-Hot Encoding - DÜZELTİLMİŞ)


In [5]:

print("\nPolars DataFrame'ler Pandas DataFrame'e dönüştürülüyor...")
# Polars DataFrame'leri Pandas'a dönüştür
train_df_pd = train_df_pl.to_pandas()
test_df_pd = test_df_pl.to_pandas()

# Kategorik sütunları belirleyelim
categorical_features = ['gender', 'province', 'religion', 'work_type', 'work_sector']

# NaN değerleri "UNKNOWN" ile dolduralım (inplace=True yerine doğrudan atama yaparak)
for col in categorical_features:
    # DÜZELTME BURADA:
    train_df_pd[col] = train_df_pd[col].fillna('UNKNOWN')
    test_df_pd[col] = test_df_pd[col].fillna('UNKNOWN')

print("Kategorik özelliklerin NaN değerleri dolduruldu (Pandas).")
print("Train DF (Pandas) Sütun Tipleri (ilk 50):")
print(train_df_pd.info(verbose=False, max_cols=50)) # verbose=False daha az çıktı verir

# Date sütununu artık ihtiyacımız yoksa drop edebiliriz
train_df_pd.drop(columns=['date'], errors='ignore', inplace=True)
test_df_pd.drop(columns=['date'], errors='ignore', inplace=True)


Polars DataFrame'ler Pandas DataFrame'e dönüştürülüyor...
Kategorik özelliklerin NaN değerleri dolduruldu (Pandas).
Train DF (Pandas) Sütun Tipleri (ilk 50):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133287 entries, 0 to 133286
Columns: 31 entries, cust_id to active_product_category_nbr_last_month
dtypes: datetime64[ms](1), float64(17), int64(7), object(5), uint32(1)
memory usage: 31.0+ MB
None


# Adım 4: Eğitim ve Test Seti Hazırlığı


In [6]:

target = 'churn'
features_to_drop = ['cust_id', 'ref_date']

X_train = train_df_pd.drop(columns=[target] + features_to_drop, errors='ignore')
y_train = train_df_pd[target]

X_test = test_df_pd.drop(columns=features_to_drop, errors='ignore')

numerical_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = [col for col in categorical_features if col in X_train.columns]

print("\nEğitim ve test setleri hazırlandı.")
print(f"Eğitim Seti Boyutu: {X_train.shape}")
print(f"Test Seti Boyutu: {X_test.shape}")
print("X_train sütunları (ilk 50):")
print(X_train.columns.tolist()[:50])


Eğitim ve test setleri hazırlandı.
Eğitim Seti Boyutu: (133287, 28)
Test Seti Boyutu: (43006, 28)
X_train sütunları (ilk 50):
['gender', 'age', 'province', 'religion', 'work_type', 'work_sector', 'tenure', 'mobile_eft_all_cnt_mean_hist', 'mobile_eft_all_amt_mean_hist', 'cc_transaction_all_amt_mean_hist', 'cc_transaction_all_cnt_mean_hist', 'active_product_category_nbr_mean_hist', 'mobile_eft_all_cnt_sum_hist', 'mobile_eft_all_amt_sum_hist', 'cc_transaction_all_amt_sum_hist', 'cc_transaction_all_cnt_sum_hist', 'active_product_category_nbr_sum_hist', 'mobile_eft_all_cnt_max_hist', 'mobile_eft_all_amt_max_hist', 'cc_transaction_all_amt_max_hist', 'cc_transaction_all_cnt_max_hist', 'active_product_category_nbr_max_hist', 'history_record_count', 'mobile_eft_all_cnt_last_month', 'mobile_eft_all_amt_last_month', 'cc_transaction_all_amt_last_month', 'cc_transaction_all_cnt_last_month', 'active_product_category_nbr_last_month']


## Adım 4.1: Veri Dengesizliği Oranını Hesapla


In [7]:
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]
print(f"Hesaplanan scale_pos_weight: {scale_pos_weight:.2f}")

Hesaplanan scale_pos_weight: 6.06


## Adım 4.2: Gelişmiş Recency ve Trend Özellikleri Ekleme


In [8]:
print("Gelişmiş Recency ve Trend özellikleri (Pandas) ekleniyor...")
epsilon = 1e-6

# Polars DF'den en son işlem tarihini alıp Pandas'a çevirelim
last_trans_dates = customer_history_pl.group_by("cust_id").agg(pl.col("date").max().alias("last_trans_date")).to_pandas()

# Ana Pandas DataFrame'leri ile birleştir
train_df_pd = pd.merge(train_df_pd, last_trans_dates, on='cust_id', how='left')
test_df_pd = pd.merge(test_df_pd, last_trans_dates, on='cust_id', how='left')

# Gün farkını hesapla
train_df_pd['days_since_last_transaction'] = (train_df_pd['ref_date'] - train_df_pd['last_trans_date']).dt.days
test_df_pd['days_since_last_transaction'] = (test_df_pd['ref_date'] - test_df_pd['last_trans_date']).dt.days

# Trend özellikleri
train_df_pd['cc_spend_trend_ratio'] = train_df_pd['cc_transaction_all_amt_last_month'] / (train_df_pd['cc_transaction_all_amt_mean_hist'] + epsilon)
test_df_pd['cc_spend_trend_ratio'] = test_df_pd['cc_transaction_all_amt_last_month'] / (test_df_pd['cc_transaction_all_amt_mean_hist'] + epsilon)

# Oluşan NaN değerleri doldur
train_df_pd.fillna(0, inplace=True)
test_df_pd.fillna(0, inplace=True)

# Gereksiz tarih sütununu kaldır
train_df_pd.drop(columns=['last_trans_date'], inplace=True)
test_df_pd.drop(columns=['last_trans_date'], inplace=True)

# ÇOK ÖNEMLİ: Yeni özellikleri ekledikten sonra X_train, X_test ve sütun listelerini yeniden oluşturmalıyız!
X_train = train_df_pd.drop(columns=[target] + features_to_drop, errors='ignore')
X_test = test_df_pd.drop(columns=features_to_drop, errors='ignore')
numerical_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = [col for col in categorical_features if col in X_train.columns]

print("Gelişmiş özellikler eklendi ve X_train/X_test/sütun listeleri güncellendi.")
print(f"Yeni Eğitim Seti Boyutu: {X_train.shape}")

Gelişmiş Recency ve Trend özellikleri (Pandas) ekleniyor...
Gelişmiş özellikler eklendi ve X_train/X_test/sütun listeleri güncellendi.
Yeni Eğitim Seti Boyutu: (133287, 30)


In [9]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import numpy as np


def recall_at_k(y_true, y_prob, k=0.1):
    """
    Tahmin edilen olasılıkların en üst k%'sını pozitif etiketleyerek recall değerini hesaplar.

    Parametreler:
        y_true (list): Gerçek ikili etiketler.
        y_prob (list): Tahmin edilen olasılıklar.
        k (float): Pozitif etiketlenecek olasılıkların yüzdelik dilimi (varsayılan 0.1).

    Döndürür:
        float: En iyi k% tahminlerindeki recall oranı.
    """
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)
    n = len(y_true)
    m = max(1, int(np.round(k * n)))
    order = np.argsort(-y_prob, kind="mergesort")
    top = order[:m]

    tp_at_k = y_true[top].sum()
    P = y_true.sum()

    return float(tp_at_k / P) if P > 0 else 0.0


def lift_at_k(y_true, y_prob, k=0.1):
    """
    Tahmin edilen olasılıkların en üst k%'sını pozitif etiketleyerek lift (precision/prevalence) değerini hesaplar.

    Parametreler:
        y_true (list): Gerçek ikili etiketler.
        y_prob (list): Tahmin edilen olasılıklar.
        k (float): Pozitif etiketlenecek olasılıkların yüzdelik dilimi (varsayılan 0.1).

    Döndürür:
        float: En iyi k% tahminlerindeki lift değeri.
    """
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)
    n = len(y_true)
    m = max(1, int(np.round(k * n)))
    order = np.argsort(-y_prob, kind="mergesort")
    top = order[:m]

    tp_at_k = y_true[top].sum()
    precision_at_k = tp_at_k / m
    prevalence = y_true.mean()

    return float(precision_at_k / prevalence) if prevalence > 0 else 0.0


def convert_auc_to_gini(auc):
    """
    ROC AUC skorunu Gini katsayısına dönüştürür.

    Gini katsayısı, ROC AUC skorunun doğrusal bir dönüşümüdür.

    Parametreler:
        auc (float): ROC AUC skoru (0 ile 1 arasında).

    Döndürür:
        float: Gini katsayısı (-1 ile 1 arasında).
    """
    return 2 * auc - 1


def ing_hubs_datathon_metric(y_true, y_prob):
    """
    Gini, recall@10% ve lift@10% metriklerini birleştiren özel bir metrik hesaplar.

    Metrik, her bir skoru bir baseline modelin metrik değerlerine göre oranlar ve aşağıdaki ağırlıkları uygular:
    - Gini: %40
    - Recall@10%: %30
    - Lift@10%: %30

    Parametreler:
        y_true (list): Gerçek ikili etiketler.
        y_prob (list): Tahmin edilen olasılıklar.

    Döndürür:
        float: Ağırlıklandırılmış bileşik skor.
    """
    # final metrik için ağırlıklar
    score_weights = {
        "gini": 0.4,
        "recall_at_10perc": 0.3,
        "lift_at_10perc": 0.3,
    }

    # baseline modelin her bir metrik için değerleri
    baseline_scores = {
        "roc_auc": 0.6925726757936908,
        "recall_at_10perc": 0.18469015795868773,
        "lift_at_10perc": 1.847159286784029,
    }

    # y_prob tahminleri için metriklerin hesaplanması
    roc_auc = roc_auc_score(y_true, y_prob)
    recall_at_10perc = recall_at_k(y_true, y_prob, k=0.1)
    lift_at_10perc = lift_at_k(y_true, y_prob, k=0.1)

    new_scores = {
        "roc_auc": roc_auc,
        "recall_at_10perc": recall_at_10perc,
        "lift_at_10perc": lift_at_10perc,
    }

    # roc auc değerlerinin gini değerine dönüştürülmesi
    baseline_scores["gini"] = convert_auc_to_gini(baseline_scores["roc_auc"])
    new_scores["gini"] = convert_auc_to_gini(new_scores["roc_auc"])

    # baseline modeline oranlama
    final_gini_score = new_scores["gini"] / baseline_scores["gini"]
    final_recall_score = new_scores["recall_at_10perc"] / baseline_scores["recall_at_10perc"]
    final_lift_score = new_scores["lift_at_10perc"] / baseline_scores["lift_at_10perc"]

    # ağırlıklandırılmış metriğin hesaplanması
    final_score = (
        final_gini_score * score_weights["gini"] +
        final_recall_score * score_weights["recall_at_10perc"] + 
        final_lift_score * score_weights["lift_at_10perc"]
    )
    return final_score

In [10]:
# XGBoost'un anlayacağı formatta metrik fonksiyonu
def xgb_metric(y_pred, y_true_dmatrix):
    y_true = y_true_dmatrix.get_label()
    score = ing_hubs_datathon_metric(y_true, y_pred)
    # XGBoost erken durdurmada metriği maksimize etmeye çalışır, bu yüzden olduğu gibi bırakıyoruz.
    return 'ING_Datathon_Metric', score

# ADIM 5 & 6 (NİHAİ VERSİYON - SON DÜZELTME): Özel Metrik ile Çapraz Doğrulama ve Tahmin


In [11]:
# ADIM 5 & 6 (SON VURUŞ - YA HEP YA HİÇ)
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

print("SON VURUŞ PLANI BAŞLATILDI: Parametreler zorlanıyor...")

# scale_pos_weight'i DAHA DA agresif yapıyoruz. Model "churn" demeye zorlanacak.
scale_pos_weight_tuned = scale_pos_weight * 1.3 # %30 artış

# Hiperparametreleri Recall ve Lift'i patlatmak için ayarlıyoruz
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'use_label_encoder': False,
    'scale_pos_weight': scale_pos_weight_tuned,
    'n_estimators': 1800, # Daha fazla ağaç
    'learning_rate': 0.015, # Daha yavaş öğren, daha hassas ol
    'random_state': 42,
    'n_jobs': -1,
    'colsample_bytree': 0.65, # Daha az özellik kullan, daha çeşitli ağaçlar yarat
    'subsample': 0.7,
    'max_depth': 7, # DAHA DERİN AĞAÇLAR. Karmaşık ilişkileri yakalamak için. Overfit riski var ama denemeye değer.
    'gamma': 0.2, # Daha fazla regularizasyon
    'lambda': 1.5, # Daha fazla regularizasyon
}

# Basit Pipeline yapısı
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', min_frequency=500), categorical_features),
    ],
    remainder='passthrough'
)

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', xgb.XGBClassifier(**xgb_params))])


print("\n'Son Vuruş' modeli tüm eğitim verisi üzerinde eğitiliyor...")
model.fit(X_train, y_train)
print("Model eğitimi tamamlandı.")

# Tahmin Yapma
print("\nTahminler yapılıyor...")
predictions = model.predict_proba(X_test)[:, 1]

# Sonuç Dosyasını Oluşturma
submission_df = pd.DataFrame({'cust_id': test_df_pd['cust_id'], 'churn': predictions})
submission_df.to_csv('submission_last_shot.csv', index=False)

print("\nTahminler yapıldı ve submission_last_shot.csv dosyası oluşturuldu.")
print("BU SON ŞANSIN! DÜŞÜNME, GÖNDER!")
print(submission_df.head())

SON VURUŞ PLANI BAŞLATILDI: Parametreler zorlanıyor...

'Son Vuruş' modeli tüm eğitim verisi üzerinde eğitiliyor...
Model eğitimi tamamlandı.

Tahminler yapılıyor...

Tahminler yapıldı ve submission_last_shot.csv dosyası oluşturuldu.
BU SON ŞANSIN! DÜŞÜNME, GÖNDER!
   cust_id     churn
0        1  0.439001
1        2  0.207924
2        9  0.417321
3       15  0.436015
4       19  0.201719
